In [99]:
from pymongo import MongoClient
from pydantic import BaseModel, Field, EmailStr, ValidationError, BeforeValidator
from datetime import datetime, timezone
from enum import Enum
from dotenv import load_dotenv
import os
load_dotenv()

True

In [100]:

client = MongoClient(os.getenv("MONGODB_URL"))
db = client[os.getenv("DB_NAME")]
historytable = db[os.getenv("COLLECTION_NAME")]

In [101]:
mydict = { "session_id": "abc123", "type": "human", "content": "Hello" }

x = list(historytable.find({}))
# docs = list(historytable.find({}))
# lean_docs = [{**doc, "_id": str(doc["_id"])} for doc in docs]
print(x)

[{'_id': ObjectId('6878a9953ece4e35531a8c41'), 'SessionId': '6880975c036e1aacd3bed0c4', 'History': '{"type": "human", "data": {"content": "india vs pakistan", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}', 'userId': 'test'}, {'_id': ObjectId('6878a9953ece4e35531a8c42'), 'SessionId': 'abtestc123', 'History': '{"type": "ai", "data": {"content": "The India vs. Pakistan rivalry, particularly in cricket, is one of the most intense and passionate rivalries in the world of sports. Matches between these two teams often attract large audiences and generate significant media coverage, reflecting the historical, political, and cultural complexities between the two countries.\\n\\n### Key Points about the Rivalry:\\n\\n1. **Historical Context**: The rivalry has deep historical roots, stemming from the partition of British India in 1947, which led to significant conflict and animosity between the two nations.\\n\\n2. **Cricketing Ri

In [102]:
#  defining structor / validate data for insertion in database 

# enum for type validation

class MessageType(str, Enum):
    human = "human"
    ai = "ai"


class chathistory(BaseModel):
    # _id: str = Field(default=object)
    session_id: str = Field(min_length=2, description="session id for the chatting")
    type: MessageType = Field(..., description="Type of message: human or ai")
    content:str              
    timestamp: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))

In [103]:
message = chathistory(**mydict)
# historytable.insert_one(message.model_dump())
# print(message.model_dump())

In [104]:
from langchain_community.chat_message_histories import MongoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_community.tools import BraveSearch
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=512)
tablehistory_collectionname = os.getenv("COLLECTION_NAME")

brave_tool = BraveSearch.from_api_key(api_key=os.getenv("BRAVE_API_KEY"), search_kwargs={"count": 20})
tools = [brave_tool]

llm_with_tools = llm.bind_tools(tools)


In [105]:
def get_memory(sessionid:str):
    mongo_url = os.getenv("MONGODB_URL")
    db_name = os.getenv("DB_NAME")
    chat_history = MongoDBChatMessageHistory(
        connection_string=mongo_url,
        session_id=sessionid,
        database_name=db_name,
        collection_name=tablehistory_collectionname
    )
    memory = ConversationBufferMemory(chat_memory=chat_history, return_messages=True)
    return chat_history


In [106]:
# prompt = ChatPromptTemplate.from_messages([
#     MessagesPlaceholder(variable_name="history"), 
#     ("human", "{input}")
# ])
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. If you don’t know the answer or if the query is about current events or the internet, call a tool."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain_with_tools = prompt | llm_with_tools | StrOutputParser()
core_chain = prompt | llm | StrOutputParser()





In [107]:
message = { "session_id": "abtestc123", "content": "what is the current score of india in india vs england" }
memory = get_memory(message["session_id"])

In [108]:
with_history_chain = RunnableWithMessageHistory(
            runnable=chain_with_tools,
            get_session_history=get_memory,
            input_messages_key="input",  
            history_messages_key="history"
        )

ai_response = await with_history_chain.ainvoke(
        {"input":message["content"]}, 
        config={
            "configurable": {"session_id": message["session_id"]}
        }
    )

# make the history output as question for brave



# get the data from brave search 

# make the output to show to user

# make history again

# send the output to user
print(ai_response)

In [110]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import BraveSearch
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_community.chat_message_histories import MongoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
import os

# 1. Setup tool-enabled LLM (n=1 by default)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 2. Define tools
brave_tool = BraveSearch.from_api_key(api_key=os.getenv("BRAVE_API_KEY"))
tools = [brave_tool]

# 3. Create a Tool-Calling prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a smart assistant. Use tools like BraveSearch when user asks about current events or unknown topics."),
    MessagesPlaceholder(variable_name="history"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("human", "{input}")
])

# 4. Create the agent using LangChain Agent Runner
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 5. Create session history using MongoDB
def get_memory(session_id: str):
    chat_history = MongoDBChatMessageHistory(
        connection_string=os.getenv("MONGODB_URL"),
        session_id=session_id,
        database_name=os.getenv("DB_NAME"),
        collection_name=os.getenv("COLLECTION_NAME")
    )
    return chat_history

# 6. Attach history to the agent
with_history_agent = RunnableWithMessageHistory(
    runnable=agent_executor,
    get_session_history=lambda sid: get_memory(sid),
    input_messages_key="input",
    history_messages_key="history"
)

# 7. Input message
message = {
    "session_id": "abtestc123",
    "content": "tell me current score of india in india vs england"
}

# 8. Run the agent with memory
ai_response = await with_history_agent.ainvoke(
    {"input": message["content"]},
    config={"configurable": {"session_id": message["session_id"]}}
)

print(ai_response['output'])




> Entering new AgentExecutor chain...
I'm unable to provide real-time scores or updates. To find the current score for the **India vs. England** match, please check reliable sports websites or apps such as:

1. **ESPN Cricinfo**: [ESPN Cricinfo](https://www.espncricinfo.com) - For live scores and detailed commentary.
2. **Cricbuzz**: [Cricbuzz](https://www.cricbuzz.com) - Offers real-time updates and match analysis.

If you have any other questions or need information about a specific match, feel free to ask!

> Finished chain.
I'm unable to provide real-time scores or updates. To find the current score for the **India vs. England** match, please check reliable sports websites or apps such as:

1. **ESPN Cricinfo**: [ESPN Cricinfo](https://www.espncricinfo.com) - For live scores and detailed commentary.
2. **Cricbuzz**: [Cricbuzz](https://www.cricbuzz.com) - Offers real-time updates and match analysis.

If you have any other questions or need information about a specific match, feel 